In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import BCEWithLogitsLoss

In [2]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, TensorDataset
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [4]:
# Define model name and number of labels
model_name = "sampathlonka/San-BERT"
num_labels = 49  # Replace with the actual number of labels in your data

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/472k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/951k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sampathlonka/San-BERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
data = pd.read_excel('/kaggle/input/prepro/manipulated_data.xlsx')

In [7]:
test_split = 0.2

# Initial train and test split.
train_df, test_df = train_test_split(
    data,
    test_size=test_split,
)
print(f"Number of rows in training set: {len(train_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 824
Number of rows in test set: 207


In [8]:
# Split the data into texts and labels
train_texts = train_df['Text'].tolist()
train_labels = train_df.iloc[:, 1:].values.tolist()

In [9]:
test_split = 0.5

# Initial train and test split.
val_df, test_df = train_test_split(
    test_df,
    test_size=test_split,
)
print(f"Number of rows in training set: {len(val_df)}")
print(f"Number of rows in test set: {len(test_df)}")

Number of rows in training set: 103
Number of rows in test set: 104


In [10]:
# Split the data into texts and labels
val_texts = val_df['Text'].tolist()
val_labels = val_df.iloc[:, 1:].values.tolist()

In [11]:
#train_texts
#train_labels

In [12]:
# Tokenize data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)

In [13]:
# Tokenize data
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

In [14]:
#train_encodings[0]
#train_encodings

In [15]:
from torch.utils.data import Dataset

In [16]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.encodings["input_ids"][idx]),
            'attention_mask': torch.tensor(self.encodings["attention_mask"][idx]),
            'labels': torch.tensor(self.labels[idx])
        }
        return item

In [17]:
# Create custom dataset
train_dataset = CustomDataset(train_encodings, train_labels)

# Create dataloader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False)

In [18]:
# Create custom dataset
val_dataset = CustomDataset(val_encodings, val_labels)

# Create dataloader
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [19]:
#next(iter(train_dataset))
#next(iter(train_dataloader))

In [20]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

In [21]:
num_epochs = 10

for epoch in range(num_epochs):
    # Training loop
    model.train()  # Set the model to training mode
    train_loss = 0.0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/ {num_epochs}", leave=False):
        input_ids = batch['input_ids'].to(device)  # Move input to device
        attention_mask = batch['attention_mask'].to(device)  # Move attention mask to device
        labels = batch['labels'].to(device)  # Move labels to device

        optimizer.zero_grad()  # Clear gradients
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = criterion(logits, labels.float())
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters

        train_loss += loss.item()

    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Validation", leave=False):
            input_ids = batch['input_ids'].to(device)  # Move input to device
            attention_mask = batch['attention_mask'].to(device)  # Move attention mask to device
            labels = batch['labels'].to(device)  # Move labels to device

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = criterion(logits, labels.float())

            val_loss += loss.item()

    # Calculate average loss
    avg_train_loss = train_loss / len(train_dataloader)
    avg_val_loss = val_loss / len(val_dataloader)

    # Print epoch statistics
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss}, Val Loss: {avg_val_loss}")

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
test_texts =test_df['Text'].tolist()
test_labels = test_df.iloc[:, 1:].values.tolist()
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)
test_dataset = CustomDataset(test_encodings, val_labels)
test_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [ ]:
#next(iter(test_dataloader))

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate(model, dataloader):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for input_ids, attention_mask, labels in dataloader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predicted_labels = torch.argmax(logits, dim=1)

            predictions.extend(predicted_labels.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    confusion = confusion_matrix(true_labels, predictions)

    # Calculate classification report
    report = classification_report(true_labels, predictions)

    print("Evaluation Metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print("Confusion Matrix:")
    print(confusion)
    print("Classification Report:")
    print(report)

    return accuracy, precision, recall, f1, confusion

In [ ]:
from huggingface_hub import notebook_login
notebook_login()